In [1]:
from transformers import BertTokenizer
 
# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
 
# Input text
text = 'ChatGPT is a language model developed by OpenAI, based on the GPT (Generative Pre-trained Transformer) architecture. '
 
# Tokenize and encode the text
encoding = tokenizer.encode(text)
 
# Print the token IDs
print("Token IDs:", encoding)
 
# Convert token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(encoding)
 
# Print the corresponding tokens
print("Tokens:", tokens)

ModuleNotFoundError: No module named 'transformers'

In [1]:
from itertools import product
from itertools import combinations

import os

import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, ConcatDataset

import random
import ast
        

import torch.nn.functional as F
#from torcheval.metrics.functional import multiclass_confusion_matrix
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

from torchvision.utils import save_image
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torchvision import transforms

import optuna

from itertools import product
from itertools import combinations


import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation ,FFMpegWriter ,PillowWriter

import dill
import numpy as np

import json
import pandas as pd
from tqdm import tqdm
import copy
import os 
import random

from collections import OrderedDict
import wandb


In [2]:
#!pip install wandb

In [3]:
wandb.login(key="ab631efc36e2c87f5f54d82b5cdbd6c501d5221f")

wandb: Currently logged in as: aymentlili. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/crns/.netrc


True

In [4]:
import accelerate

In [5]:
#!accelerate config

In [6]:
random.seed(881)

In [7]:
from Double_input_transformer import CustomDataset,TransformerAE

# Trainloader code

In [8]:

# Load the list
train_pair = np.load('train_pair_seed881.npy', allow_pickle=True)
test_pair = np.load('test_pair_seed881.npy', allow_pickle=True)
val_pair = np.load('val_pair_seed881.npy', allow_pickle=True)


In [9]:
def batchify(lst, batch_size):
    return [lst[i:i+batch_size] for i in range(0, len(lst), batch_size)]

# Example usage:
my_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
batch_size = 4
batches = batchify(my_list, batch_size)
print(batches)

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10]]


In [10]:
#cs_tr=CustomDataset(train_pair,batch_size=200)

In [11]:
#Dataset,EXP,ACC,U = cs_tr[0]
#x1,x2,tg = Dataset[:,0,:], Dataset[:,1,:],Dataset[:,2,:]

In [12]:
#x1.shape

# Autoencoder

In [13]:
#!pip install einops

In [14]:
class EmbedderNeuronGroup(nn.Module):
    def __init__(self, d_model, seed=22):
        super().__init__()
        #print("EmbedderNeuroneGroup")
        self.neuron_l1 = nn.Linear(200, d_model) #8
        self.neuron_l2 = nn.Linear(72, d_model) #12

    def forward(self, x):
        return self.multiLinear(x)

    def multiLinear(self, v):
        #print("multi-linear method",v.shape)

        l = []

        for ndx in range(8):
            idx_start = ndx * 200
            idx_end = idx_start + 200
            l.append(self.neuron_l1(v[:,idx_start:idx_end]))

        # l2
        for ndx in range(12):
            idx_start = 200*8 + ndx * 72
            idx_end = idx_start + 72
            l.append(self.neuron_l2(v[:,idx_start:idx_end]))
        #print(len(l))
        #print(len(l[0]))
        final = torch.stack(l, dim=1)

        # print(final.shape)
        return final



In [15]:
# max_seq_len=176,
# N=4
# heads=3
# d_model=900
# d_ff=900
# neck=700
# dropout=0.1
# # Enc=EncoderNeuronGroup(d_model=d_model, N=N, heads=heads, max_seq_len=max_seq_len, dropout=dropout,d_ff=d_ff)
# # vec1 = torch.rand(1,2464)
# # res,scores=Enc(vec1)
# # res.shape

In [16]:
# vec2neck = nn.Linear(d_ff*2, neck)
# print(res.shape)
# out3=torch.cat([res,res], dim=2)
# print("neck input:",out3.shape)
# sum_r=torch.sum(out3, dim=1, keepdim=False)
# vec2=vec2neck(sum_r)
# print(len(vec2))
# tanh = nn.Tanh()
# neck_t=tanh(vec2)
# print("neck shape:",neck_t.shape)

In [17]:
# Dec=DecoderNeuronGroup(d_model=d_model, N=N, heads=heads, max_seq_len=max_seq_len, dropout=dropout,d_ff=d_ff,neck=neck)
# res,scores=Dec(neck_t)
# res.shape

In [18]:
len(test_pair),len(train_pair),len(val_pair)#,len(test_tgt),len(train_tgt),len(val_tgt)

(159516, 595620, 88560)

In [19]:


# print(mod.numParams())
# x1 = torch.rand(1,2464)
# x2 = torch.rand(1,2464)
# mod=mod.to(device).to(torch.float32)

# #x1=x1.to(torch.float32)
# #x2=x2.to(torch.float32)
# x1=x1.to(device)
# x2=x2.to(device)
# mod=mod.to(device)
# out = mod(x1,x2)
# print("Output Shape: ", out[0].shape)

# from torchinfo import summary

# summary(mod)


In [20]:
from torchvision import datasets
IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

class ClassSpecificImageFolder(datasets.DatasetFolder):
    def __init__(
            self,
            root,
            dropped_classes=[],
            transform = None,
            target_transform = None,
            loader = datasets.folder.default_loader,
            is_valid_file = None,
    ):
        self.dropped_classes = dropped_classes
        super(ClassSpecificImageFolder, self).__init__(root, loader, IMG_EXTENSIONS if is_valid_file is None else None,
                                                       transform=transform,
                                                       target_transform=target_transform,
                                                       is_valid_file=is_valid_file)
        self.imgs = self.samples

    def find_classes(self, directory):
        classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())
        classes = [c for c in classes if c not in self.dropped_classes]
        if not classes:
            raise FileNotFoundError(f"Couldn't find any class folder in {directory}.")

        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx

#from torcheval.metrics.functional import multiclass_confusion_matrix

class CNN(nn.Module):
    def __init__(
        self,
        channels_in,
        nlin="leakyrelu",
        dropout=0.0,
        init_type="uniform",
    ):
        super().__init__()
        # init module list
        self.module_list = nn.ModuleList()
        ### ASSUMES 28x28 image size
        ## compose layer 1
        self.module_list.append(nn.Conv2d(channels_in, 8, 5))
        self.module_list.append(nn.MaxPool2d(2, 2))
        self.module_list.append(self.get_nonlin(nlin))
        # apply dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## compose layer 2
        self.module_list.append(nn.Conv2d(8, 6, 5))
        self.module_list.append(nn.MaxPool2d(2, 2))
        self.module_list.append(self.get_nonlin(nlin))
        ## add dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## compose layer 3
        self.module_list.append(nn.Conv2d(6, 4, 2))
        self.module_list.append(self.get_nonlin(nlin))
        ## add flatten layer
        self.module_list.append(nn.Flatten())
        ## add linear layer 1
        self.module_list.append(nn.Linear(3 * 3 * 4, 20))
        self.module_list.append(self.get_nonlin(nlin))
        ## add dropout
        if dropout > 0:
            self.module_list.append(nn.Dropout(dropout))
        ## add linear layer 1
        self.module_list.append(nn.Linear(20, 10))

        ### initialize weights with se methods
        self.initialize_weights(init_type)

    def initialize_weights(self, init_type):
        # print("initialze model")
        for m in self.module_list:
            if type(m) == nn.Linear or type(m) == nn.Conv2d:
                if init_type == "xavier_uniform":
                    torch.nn.init.xavier_uniform_(m.weight)
                if init_type == "xavier_normal":
                    torch.nn.init.xavier_normal_(m.weight)
                if init_type == "uniform":
                    torch.nn.init.uniform_(m.weight)
                if init_type == "normal":
                    torch.nn.init.normal_(m.weight)
                if init_type == "kaiming_normal":
                    torch.nn.init.kaiming_normal_(m.weight)
                if init_type == "kaiming_uniform":
                    torch.nn.init.kaiming_uniform_(m.weight)
                # set bias to some small non-zero value
                m.bias.data.fill_(0.01)

    def get_nonlin(self, nlin):
        # apply nonlinearity
        if nlin == "leakyrelu":
            return nn.LeakyReLU()
        if nlin == "relu":
            return nn.ReLU()
        if nlin == "tanh":
            return nn.Tanh()
        if nlin == "sigmoid":
            return nn.Sigmoid()
        if nlin == "silu":
            return nn.SiLU()
        if nlin == "gelu":
            return nn.GELU()


    def forward(self, x):
        # forward prop through module_list
        for layer in self.module_list:
            x = layer(x)
        return x

    def forward_activations(self, x):
        # forward prop through module_list
        activations = []
        for layer in self.module_list:
            x = layer(x)
            if (
                isinstance(layer, nn.Tanh)
                or isinstance(layer, nn.Sigmoid)
                or isinstance(layer, nn.ReLU)
                or isinstance(layer, nn.LeakyReLU)
                or isinstance(layer, nn.SiLU)
                or isinstance(layer, nn.GELU)
                or isinstance(layer, ORU)
                or isinstance(layer, ERU)
            ):
                activations.append(x)
        return x, activations
def train(model, trainloader, optimizer, criterion,nb_classes):
    List_mx=[]
    model.train()
    #print('Training')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in enumerate(trainloader):
        counter += 1
        image, labels = data
        image = image
        labels = labels
        optimizer.zero_grad()
        # forward pass
        outputs = model(image)
        # calculate the loss
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        # calculate the accuracy
        _, preds = torch.max(outputs.data, 1)
        train_running_correct += (preds == labels).sum().item()
        #mx=multiclass_confusion_matrix(preds ,labels,nb_classes,normalize="pred")
        #List_mx.append(mx)
        # backpropagation
        loss.backward()
        # update the optimizer parameters
        optimizer.step()
    
    # loss and accuracy for the complete epoch
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc,List_mx


def validate(model, testloader, criterion,nb_classes):
    List_mx=[]
    model.eval()
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in enumerate(testloader):
            if data is None:  # Skip None values
                continue
            counter += 1
            
            image, labels = data
            image = image
            labels = labels
            # forward pass
            outputs = model(image.to(torch.float32))
            # calculate the loss
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            # calculate the accuracy
            _, preds = torch.max(outputs.data, 1)
            valid_running_correct += (preds == labels).sum().item()
            #mx=multiclass_confusion_matrix(preds ,labels,nb_classes,normalize="pred")
            #List_mx.append(mx)
        
    # loss and accuracy for the complete epoch
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc,List_mx
def create_frame(step,ax,data):
    ax=ax.cla()
    sns.heatmap(data[step][-1].cpu(),annot=True,cmap="cubehelix",ax=ax,cbar=False)
    plt.title('Epoch {} training {}'.format(step,exp)  )

In [21]:
#L_activations=["gelu","relu","silu","leakyrelu","sigmoid","tanh"]
#csv_files,L_activations

In [22]:
#(2) in https://arxiv.org/pdf/2209.14733.pdf
vec1 = torch.rand(1,2464)
vec2 = torch.rand(1,2464)
class LWLN_loss(nn.Module):
    def __init__(self):
        super(LWLN_loss, self).__init__()
    def forward(self, vec1,vec2):
        loss = (torch.mean((vec1[:,0:208]-vec2[:,0:208])**2)/vec2[:,0:208].std() + 
                 torch.mean((vec1[:,208:1414]-vec2[:,208:1414])**2)/vec2[:,208:1414].std()+ 
                 torch.mean((vec1[:,1414:1514]-vec2[:,1414:1514])**2)/vec2[:,1414:1514].std()+
                 torch.mean((vec1[:,1514:2254]-vec2[:,1514:2254])**2)/vec2[:,1514:2254].std()+
                 torch.mean((vec1[:,2254:2464]-vec2[:,2254:2464])**2)/vec2[:,2254:2464].std())/(6)
        
        return loss
LW=LWLN_loss()
LW(vec1,vec2)

tensor(0.5047)

In [23]:
Cols=["label task 1","index","Accuracy task1",\
      "label task 2","Accuracy task2"]+ \
["weight {}".format(x) for x in range(200)]+["bias {}".format(x) for x in range(200,208)]+ \
["weight {}".format(x) for x in range(208,1408)]+["bias {}".format(x) for x in range(1408,1414)]+ \
["weight {}".format(x) for x in range(1414,1510)]+["bias {}".format(x) for x in range(1510,1514)]+ \
["weight {}".format(x) for x in range(1514,2234)]+["bias {}".format(x) for x in range(2234,2254)]+ \
["weight {}".format(x) for x in range(2254,2454)]+["bias {}".format(x) for x in range(2454,2464)]+ \
["Loader Set","Reconstructed Accuracy ID","Actual Accuracy","Reconstructed Accuracy OOD","Transformer Loss","lr",'epochCNN','ActivationCNN'] 

print(len(Cols))
predicted_Weights= pd.DataFrame(columns=Cols)

# row=["".format(task1),int(ind[0]),ACC[0],"".format(task2),ACC[1]]+vector_aux.to_list()+["train",valid_epoch_acc0,ACC[2],valid_epoch_acc1,L_train[-1]]
# predicted_Weights.append(row, ignore_index=True)
predicted_Weights

2477


,label task 1,index,Accuracy task1,label task 2,Accuracy task2,weight 0,weight 1,weight 2,weight 3,weight 4,...,bias 2462,bias 2463,Loader Set,Reconstructed Accuracy ID,Actual Accuracy,Reconstructed Accuracy OOD,Transformer Loss,lr,epochCNN,ActivationCNN


# Training 

In [24]:
def optimizer_to(optim, device):
    for param in optim.state.values():
        # Not sure there are any global tensors in the state dict
        if isinstance(param, torch.Tensor):
            param.data = param.data.to(device)
            if param._grad is not None:
                param._grad.data = param._grad.data.to(device)
        elif isinstance(param, dict):
            for subparam in param.values():
                if isinstance(subparam, torch.Tensor):
                    subparam.data = subparam.data.to(device)
                    if subparam._grad is not None:
                        subparam._grad.data = subparam._grad.data.to(device)
def scheduler_to(sched, device):
    for param in sched.__dict__.values():
        if isinstance(param, torch.Tensor):
            param.data = param.data.to(device)
            if param._grad is not None:
                param._grad.data = param._grad.data.to(device)

In [25]:
import datetime
print(datetime.datetime.now())

2024-06-10 15:24:47.852491


In [26]:
from torch.autograd import Variable
def loss_Contractive(W, x, recons_x, h, lam):
    dh = h * (1 - h) 

    w_sum = torch.sum(Variable(W)**2, dim=1)

    w_sum = w_sum.unsqueeze(1) # shape N_hidden x 1
 
    contractive_loss = torch.sum(torch.mm(dh**2, w_sum), 0)

    return contractive_loss.mul_(lam)

vec1 = torch.rand(1,2464)
vec2 = torch.rand(1,2464)
#print(out[1].shape,W.shape)
# for name, param in mod.named_parameters():
#     if name == 'vec2neck.weight':
#         W = param
#         break
# CL=loss_Contractive(W,vec1,vec2, out[1], 0.005)

# CL

In [27]:
def define_model(trial):
    head=trial.suggest_int("heads",4,4)
    #d_model=trial.suggest_int("d_model",200,1200)
    neck=trial.suggest_int("neck",700,700)
    p=0.237#trial.suggest_float("dropout",0.2,0.4)
    mod = TransformerAE(max_seq_len=20,
                        N=head,
                        heads=4,
                        d_model=1200,
                        d_ff=1200,
                        neck=neck,
                        dropout=p
                       )
    print( "model params: ",head,neck,p)
    return mod,head,neck,p

In [28]:
# Turn off optuna log notes.
optuna.logging.set_verbosity(optuna.logging.WARN)


def logging_callback(study, frozen_trial):
    previous_best_value = study.user_attrs.get("previous_best_value", None)
    if previous_best_value != study.best_value:
        study.set_user_attr("previous_best_value", study.best_value)
        print(
            "Trial {} finished with best value: {} and parameters: {}. ".format(
            frozen_trial.number,
            frozen_trial.value,
            frozen_trial.params,
            )
        )

In [29]:
### Imports
from collections import deque
from typing import Dict, Optional, Literal
import torch
import torch.nn as nn


### Grokfast
def gradfilter_ema(
    m: nn.Module,
    grads: Optional[Dict[str, torch.Tensor]] = None,
    alpha: float = 0.99,
    lamb: float = 5.0,
) -> Dict[str, torch.Tensor]:
    if grads is None:
        grads = {n: p.grad.data.detach() for n, p in m.named_parameters() if p.requires_grad}

    for n, p in m.named_parameters():
        if p.requires_grad:
            grads[n] = grads[n] * alpha + p.grad.data.detach() * (1 - alpha)
            p.grad.data = p.grad.data + grads[n] * lamb

    return grads


### Grokfast-MA
def gradfilter_ma(
    m: nn.Module,
    grads: Optional[Dict[str, deque]] = None,
    window_size: int = 128,
    lamb: float = 5.0,
    filter_type: Literal['mean', 'sum'] = 'mean',
    warmup: bool = True,
    trigger: bool = False,
) -> Dict[str, deque]:
    if grads is None:
        grads = {n: deque(maxlen=window_size) for n, p in m.named_parameters() if p.requires_grad}

    for n, p in m.named_parameters():
        if p.requires_grad:
            grads[n].append(p.grad.data.detach())

            if not warmup or len(grads[n]) == window_size and not trigger:
                if filter_type == "mean":
                    avg = sum(grads[n]) / len(grads[n])
                elif filter_type == "sum":
                    avg = sum(grads[n])
                else:
                    raise ValueError(f"Unrecognized filter_type {filter_type}")
                p.grad.data = p.grad.data + avg * lamb

    return grads

In [31]:
import random
import csv 
import time
import sys
import time
import datetime
import warnings
import traceback
import gc
from accelerate import Accelerator
global track

track=0

from torchinfo import summary

# 

warnings.filterwarnings("ignore")


from optuna.storages import JournalStorage, JournalFileStorage

storage = JournalStorage(JournalFileStorage("optuna-journal DDP 3 Losses.log"))

def objective(trial):
    global track
    grads=None
    accelerator = Accelerator()
    Loss=trial.suggest_categorical("Loss function",["MSE"])#,"LWLN","Contractive",])
    Lambda=0
    if Loss=="MSE":
        results_path="/media/crns/ADATA HD330/Experiments/model MSE/"
    if Loss=="Contractive":
        results_path="./Contractive model/"
        Lambda=trial.suggest_float("Contractive_Lambda",0.00001,0.0001)
    if Loss=="LWLN":
        results_path="./mixed model/"
        
    cnn_acc_ID=[]
    cnn_acc_OOD=[]
    
    step_size=0
    factor=0
    threshhold=0
    threshold_mode = 0
    eps=0
   
    
    mod,head,neck,p=define_model(trial)
    print(summary(mod))
    lr=0.6 #trial.suggest_float("Learning_rate",0.0002,0.5)
    optimizer = Adam(mod.parameters(), lr=lr,eps=1e-10,weight_decay=0.005)
    sched_name=trial.suggest_categorical("scheduler",["CyclicLR"])#,"ReduceLROnPlateau"])
    if sched_name=="CyclicLR" :
        step_size=120000#trial.suggest_int("step_size_up",900,2800)
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=1e-3, max_lr=0.9, step_size_up=step_size, step_size_down=80000,scale_mode="iterations",mode="triangular2", cycle_momentum=False)
    if sched_name=="ReduceLROnPlateau" :
        factor=trial.suggest_float("R-lr-OP_factor",0.001,0.5)
        threshhold=trial.suggest_float("R-lr-OP_threshhold",0.0001,0.001)
        threshold_mode = trial.suggest_categorical("thresh_mod",["rel","abs"])
        eps=trial.suggest_float("R-lr-OP_eps",1e-08,1e-05)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=factor, patience=5, threshold=threshhold, threshold_mode=threshold_mode, cooldown=2, min_lr=0, eps=eps)
    
    
    batch_size=150#trial.suggest_int("batch_size",150)
    
    criterion = nn.MSELoss()
    LW=LWLN_loss()
    
    num_epochs=101
    #device = accelerator.device
    print(Loss, lr,sched_name,batch_size)
 
    run = wandb.init(
    # Set the project where this run will be logged
    project="aymen-project",
    name= "Epochs "+str(num_epochs)+" "+Loss+" "+sched_name+" "+str(batch_size),
    # Track hyperparameters and run metadata
    config={
        "Loss":Loss,
        "learning_rate": lr,
        "epochs": num_epochs,
        "head": head,
        "neck": neck,
        "p": p,
        "sched_name": sched_name,
        "step_size": step_size,
        "batch_size": batch_size,
        "step_size":step_size,
        "factor":factor,
        "threshhold":threshhold,
        "threshold_mode":threshold_mode,
        "eps":eps,
        "Lambda":Lambda
    },
)
    
    cs_tr=CustomDataset(train_pair,batch_size=batch_size)
    nb_batches = len(cs_tr)//batch_size

    #optimizer_to(optimizer,device)
    #scheduler_to(scheduler,device)

    mod, optimizer, cs_tr, scheduler = accelerator.prepare(mod, optimizer, cs_tr, scheduler)

    mod.train()
    for epoch in range(num_epochs):
        #start_time_epoch = time.time()
        for i in tqdm(range(nb_batches)):
            #start_time_batch = time.time()
            Dataset,EXP,ACC,U = cs_tr[i]
            x1,x2,tg = Dataset[:,0,:], Dataset[:,1,:],Dataset[:,2,:]
            try:
                if (Loss=="Contractive") and (i%50==0):
                    for name, param in mod.named_parameters():
                        if name == 'vec2neck.weight':
                            W = param
                            break
                       

                
                x1=x1.cuda() #.to(torch.float32)
                x2=x2.cuda() #.to(torch.float32)
                tg=tg.cuda() #.to(torch.float32)

                optimizer.zero_grad()

                output = mod(x1,x2)
                if Loss=="MSE":
                    loss_tr = criterion(output[0],tg)
                if Loss=="Contractive":
                    CL=loss_Contractive(W,vec1,vec2, output[1], Lambda)
                    loss_tr = criterion(output[0],tg)+CL
                if Loss=="LWLN":
                    loss_tr = criterion(output[0],tg)+LW(vec1,vec2)
               
               
            except Exception as e:
                # Print the exception
                print("An exception occurred:", e)
                traceback.print_exc()
                # Continue with the loop
                continue

            
            accelerator.backward(loss_tr)
            grads = gradfilter_ema(mod, grads=grads, alpha=0.98, lamb=2)
            optimizer.step()
            if sched_name=="ReduceLROnPlateau" :
                scheduler.step(loss_tr)
            else:
                 scheduler.step()

            torch.cuda.empty_cache()
            loss_to_save = float(loss_tr.detach().cpu().item())
            wandb.log({"Loss":loss_to_save})
            wandb.log({"CNN_IID":(np.mean(cnn_acc_ID))})
            wandb.log({"CNN_OOD":(np.mean(cnn_acc_OOD))})
            wandb.log({"lr": lr})    
            
        if i in [int(nb_batches/4),int(nb_batches/4)*2,int(nb_batches/4)*3,nb_batches-1] :
            cnn_acc_ID=[]
            cnn_acc_OOD=[]
            for vect in tqdm(range(len(x1))):
                y_pred=torch.unsqueeze(output[0][vect], 0) 
                y =torch.unsqueeze(tg[vect], 0) 
                
                selected_row = cs_tr.df.iloc[int(U[vect][0]), 11:17]  
                columns_with_one = selected_row[selected_row == 1].index.tolist()
                activ=columns_with_one
                epochCNN=cs_tr.df.loc[int(U[vect][0])]['epoch']
                
                
                checkpoint=OrderedDict()
                vector_aux= output[0][vect].detach()
                y_pred=vector_aux.cpu()
    
                task1=[int(x) for x in EXP[vect][0]]
                task2=[int(x) for x in EXP[vect][1]]
                task3=sorted(task1+task2)
    
    
                All=list(range(10))
                L2=[k for k in All if k not in task3] #Classes to test on (In distribution)
                L_others=[k for k in All if k not in task3] # Out of distribution classes
    
                checkpoint["module_list.0.weight"]=torch.tensor(np.array(y_pred[0:200]).reshape([8, 1, 5, 5]))
                checkpoint["module_list.0.bias"]=torch.tensor(np.array(y_pred[200:208]).reshape([8]))
    
                checkpoint["module_list.3.weight"]=torch.tensor(np.array(y_pred[208:1408]).reshape([6, 8, 5, 5]))
                checkpoint["module_list.3.bias"]=torch.tensor(np.array(y_pred[1408:1414]).reshape([6]))
    
                checkpoint["module_list.6.weight"]=torch.tensor(np.array(y_pred[1414:1510]).reshape([4, 6, 2, 2]))
                checkpoint["module_list.6.bias"]=torch.tensor(np.array(y_pred[1510:1514]).reshape([4]))
    
                checkpoint["module_list.9.weight"]=torch.tensor(np.array(y_pred[1514:2234]).reshape([20,36]))
                checkpoint["module_list.9.bias"]=torch.tensor(np.array(y_pred[2234:2254]).reshape([20]))
    
                checkpoint["module_list.11.weight"]=torch.tensor(np.array(y_pred[2254:2454]).reshape([10,20]))
                checkpoint["module_list.11.bias"]=torch.tensor(np.array(y_pred[2454:2464]).reshape([10]))
    
                Brain = CNN(1,activ[0],0,"kaiming_uniform")
    
                model=copy.deepcopy(Brain).to(torch.float32)
                model.load_state_dict(checkpoint)
    
                criterion_CNN0=CrossEntropyLoss()
    
                test_IF0=ClassSpecificImageFolder( root="./data/SplitMnist/test/",dropped_classes=[str(x) for x in L2],transform=transforms.Compose([ transforms.ToTensor(),transforms.Grayscale(1)]))
                Ts_DL0 = DataLoader(dataset=test_IF0, batch_size=20, num_workers=0, shuffle=True)
                
                _, valid_epoch_acc0,_= validate(model, Ts_DL0,  criterion_CNN0,10)
                if len(task3)==10:
                    valid_epoch_acc1=valid_epoch_acc0
                    continue
                else:
                    criterion_CNN1=CrossEntropyLoss()
                    test_IF1=ClassSpecificImageFolder( root="./data/SplitMnist/test/",dropped_classes=[str(x) for x in task3],transform=transforms.Compose([ transforms.ToTensor(),transforms.Grayscale(1)]))
                    Ts_DL1 = DataLoader(dataset=test_IF1, batch_size=20, num_workers=0, shuffle=True)
                
                valid_epoch_loss0, valid_epoch_acc1,L_mx= validate(model, Ts_DL1,  criterion_CNN1,10)
                cnn_acc_ID.append(valid_epoch_acc0)
                cnn_acc_OOD.append(valid_epoch_acc1)
                lr = optimizer.param_groups[0]["lr"]
                
        
        if epoch%10==0:
            torch.save({'epoch':epoch,'model_state_dict': mod.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'Batch Loss':loss_tr.detach().cpu().item()},results_path+'AE epoch {} {}.pth'.format(track,epoch))
    # if trial.should_prune():
    #     raise optuna.exceptions.TrialPruned()
    wandb.finish()
    track=+1
    return loss_tr
study= optuna.create_study(direction="minimize",storage=storage)
study.optimize(objective,n_trials=15,callbacks=[lambda study, trial: gc.collect()]+[logging_callback])

encoder droupout init 0.237
encoder droupout init 0.237
decoder droupout init 0.237
model params:  4 700 0.237
Layer (type:depth-idx)                        Param #
TransformerAE                                 --
├─EncoderNeuronGroup: 1-1                     --
│    └─EmbedderNeuronGroup: 2-1               --
│    │    └─Linear: 3-1                       241,200
│    │    └─Linear: 3-2                       87,600
│    └─PositionalEncoder: 2-2                 --
│    └─ModuleList: 2-3                        --
│    │    └─EncoderLayer: 3-3                 8,652,000
│    │    └─EncoderLayer: 3-4                 8,652,000
│    │    └─EncoderLayer: 3-5                 8,652,000
│    │    └─EncoderLayer: 3-6                 8,652,000
│    └─Norm: 2-4                              2,400
├─EncoderNeuronGroup: 1-2                     --
│    └─EmbedderNeuronGroup: 2-5               --
│    │    └─Linear: 3-7                       241,200
│    │    └─Linear: 3-8                       87,600
│ 

 13%|█▎        | 517/3970 [13:56<1:33:05,  1.62s/it]
[W 2024-06-13 12:31:16,843] Trial 0 failed with parameters: {'Loss function': 'MSE', 'heads': 4, 'neck': 700, 'scheduler': 'CyclicLR'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/crns/anaconda3/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_489799/3562222122.py", line 112, in objective
    Dataset,EXP,ACC,U = cs_tr[i]
  File "/home/crns/Documents/GitHub/Federated-Continual-learning-/New/Double_input_transformer.py", line 97, in __getitem__
    rowk=self.df[(self.df["label"]=='{}'.format(tg))&(self.df["epoch"]==int(self.D_epoch[str(batch[i][2])]))&(self.df[self.D_activ[str(batch[i][3])]]==float(1))]
  File "/home/crns/anaconda3/lib/python3.9/site-packages/pandas/core/ops/common.py", line 70, in new_method
    return method(self, other)
  File "/home/crns/anaconda3/lib/python3.9/site-packag

KeyboardInterrupt: 

In [ ]:
import torch
print(torch.cuda.nccl.version())

In [ ]:
x1.device 

In [ ]:
,EXP,ACC,U

In [ ]:
if epoch%30==0 or (epoch in [1,3,5,10]):
        

            
            
            vectstring=f"{y_pred.tolist()}".replace(" ", "")[1:-1].split(',')
            row_part=f"{task1};{int(U[vect][0])};{float(ACC[vect][0])};{task2};{float(ACC[vect][1])}".split(";")
            row_part2=f"train,{valid_epoch_acc0},{float(ACC[vect][2])},{valid_epoch_acc1},{float(loss_tr.detach().cpu())},{lr},{epochCNN},{activ[0]}".split(",")
            row=row_part+vectstring+row_part2
            
            temp_file = os.path.join(results_path,f'AE Tracking {track}.csv')
        
            if vect%150==0:
                print(task3,len(Ts_DL0.dataset))
                print(L2,len(Ts_DL1.dataset))
                print("epoch" ,epoch, "\t", "\t batch: ",f"/{len(Dataset)}",f"\t Learning rate :" ,f"{lr:.8f}")
                temp_file = os.path.join(results_path, f'AE Tracking {track}.csv')
                with open(temp_file, mode='a', newline='\n') as file:
                    writer = csv.writer(file)
                    writer.writerow(Cols)
                
            with open(temp_file, mode='a', newline='\n') as file:
                writer = csv.writer(file)
                writer.writerow(row)
            if vect%150==0:
                track += 1

        csv_files = [file for file in os.listdir(results_path) if file.endswith(".csv")]
        csv_files.sort(key=lambda x: int(x.split(" ")[2][:-4]))
        
        df = pd.read_csv(os.path.join(results_path, "AE Tracking 0.csv"),header=[0])
        res= pd.DataFrame(columns=df.columns)
        for file in csv_files:
            df = pd.read_csv(os.path.join(results_path, file), header=[0])
            res = res.append(df, ignore_index=True)
        print("epoch",epoch,'\n','worst_CNN_ACC \t',  float(min(res[["Reconstructed Accuracy ID"]].values)),'\n','best_CNN_ACC \t',float(max(res[["Reconstructed Accuracy ID"]].values)),'\n','max_Transformer_loss \t',float(max(res[["Transformer Loss"]].values)),'\n','min_CNN_ACC \t',float(min(res[["Transformer Loss"]].values)))
                torch.save({'epoch':epoch,'model_state_dict': mod.state_dict(),'optimizer_state_dict': optimizer.state_dict(),'Batch Loss':loss_tr.detach().cpu().item(),'last_batch_CNN_ACC':valid_epoch_acc0,'worst_CNN_ACC':  float(min(res[["Reconstructed Accuracy ID"]].values)),'best_CNN_ACC':float(max(res[["Reconstructed Accuracy ID"]].values)),'max_Transformer_loss':float(max(res[["Transformer Loss"]].values)),'min_CNN_ACC':float(min(res[["Transformer Loss"]].values))},results_path+'AE epoch {}.pth'.format(epoch))

In [ ]:
#https://discuss.pytorch.org/t/memory-management-using-pytorch-cuda-alloc-conf/157850
#https://stackoverflow.com/questions/73747731/runtimeerror-cuda-out-of-memory-how-can-i-set-max-split-size-mb

import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))